### Drop in a `config.json` for Hugging Face API Key

Please create a `config.json` file in this directory with the following content:

```json
{
  "API_KEY": "paste_api_key_here"
}
```

You are an advanced virtual AI HVAC assistant that specializes in fault detection on telemetry data that is captured from the BACnet network for different types of HVAC systems. Faults can include sensor's reading inappropriate values, energy inefficient operations, IAQ issues, and any other common issues human building operators notice when viewing graphics. Your goal is to help the human in spotting issues and then helping the human user in understanding how the HVAC is currently operating, how the HVAC is supposed to operate correctly if there is a fault detected and in a sentence explain how basic mechanical engineering theory in HVAC operations such thermodynamics, fluid mechanics, heat transfer, and any other physics for the system can be applied to the task at hand if applicable. Please indicate if you see a fault or no fault in the final output to the human. Look for faults related to fan static pressure control, temperature sensors that would be out of the range of normal values given current operating conditions, or heating/cooling value positions at maximum positions with little or no change system effects.
“Is my variable volume AHU system in a fault condition? My return air temperature is 73.9 °F, outside air is 88.1 °F, mix air is 60.4 °F, discharge air is 55.9 °F, and discharge air setpoint is 55 °F. The supply duct static pressure is 0.5 inches WC and the duct static setpoint 1.0 inches WC, and supply Fan VFD speed 100%. The outside air damper positions are at 20%, heating valve is 0%, and the cooling valve is 66%.”

In [70]:
import requests
import json

# Load configuration from file
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

# Retrieve the API key from the configuration
API_KEY = config.get("API_KEY")

# Ensure the API key is available
if not API_KEY:
    raise ValueError("API key not found in environment variables")

# Set up the API URL and headers
API_URL = "https://api-inference.huggingface.co/models/TheBloke/Llama-2-70B-Chat-GPTQ"
headers = {"Authorization": f"Bearer {API_KEY}"}

# Example telemetry data (instantaneous readings)
duct_static_pressure = 0.3
duct_static_setpoint = 0.4
fan_vfd_speed = 96

# Define the prompt
system_prompt = (
    "Always assist with care, respect, and truth. Respond with utmost utility yet securely. "
    "Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity. "
    "You are an advanced virtual AI HVAC assistant specialized in fault detection telemetry that is helping the user who is a human building operator whom needs to be taught in understanding a fault if there is one in basic mechanical engineering theory."
)

user_message = (
    f"Is my variable volume fan system in a fault? This is the supply air duct static pressure: {duct_static_pressure} in inches WC, "
    f"duct static setpoint: {duct_static_setpoint} in inches WC, and supply Fan VFD Speed: {fan_vfd_speed} speed in %."
)

# Construct the full prompt according to the required structure
prompt = (
    f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_message} [/INST]"
)

# Function to generate a response using the API
def generate_response(prompt):
    # Prepare the payload
    data = {
        "inputs": prompt,
        "parameters": {
            "max_length": 200,  # Adjust max_length for concise responses
            "temperature": 0.2,  # Adjust the creativity of the response
            "top_p": 0.95,  # Use nucleus sampling
            "num_return_sequences": 1  # Return only one response
        }
    }

    # Make the request to the Hugging Face Inference API
    response = requests.post(API_URL, headers=headers, json=data)

    # Debugging: print status code and response content
    print(f"Status Code: {response.status_code}")
    print(f"Response Content: {response.text}")

    try:
        response_json = response.json()
    except ValueError:
        print("Failed to decode JSON response")
        return ""

    # Ensure the response contains generated text
    if response.status_code == 200 and isinstance(response_json, list) and "generated_text" in response_json[0]:
        return response_json[0].get("generated_text", "").strip()
    else:
        print("Error:", response_json)
        return ""

print(prompt)
print()
print()
# Generate and print the response
assistant_reply = generate_response(prompt)
print("Assistant:", assistant_reply)


<s>[INST] <<SYS>>
Always assist with care, respect, and truth. Respond with utmost utility yet securely. Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity. You are an advanced virtual AI HVAC assistant specialized in fault detection telemetry that is helping the user who is a human building operator whom needs to be taught in understanding a fault if there is one in basic mechanical engineering theory.
<</SYS>>

Is my variable volume fan system in a fault? This is the supply air duct static pressure: 0.3 in inches WC, duct static setpoint: 0.4 in inches WC, and supply Fan VFD Speed: 96 speed in %. [/INST]


Status Code: 400
Response Content: {"error":"Model requires a Pro subscription; check out hf.co/pricing to learn more. Make sure to include your HF token in your query."}
Error: {'error': 'Model requires a Pro subscription; check out hf.co/pricing to learn more. Make sure to include your HF token in your query.'}
Assistant: 
